#### Задача 1 (10 баллов).

Попробуем себя в решении задачи определения темы текста. Будем считать, что два текста похожи по теме, если у них больше общих слов (только не предлогов с союзами), чем у других текстов. У нашей программы для определения темы будет несколько готовых текстов (достаточно больших!) с уже известной темой в базе: выберите тексты (и темы) самостоятельно, 5-6 будет достаточно.

Что должна делать программа? При запуске вы ей сообщаете название нового файла с текстом, который нужно классифицировать, она его открывает, обрабатывает и сравнивает с текстами в своей базе. С которым из текстов оказалось больше всего общих слов, того и тема! Очевидно, вам понадобится какие-то слова из текстов отбрасывать (подумайте, каким образом это сделать - здесь на самом деле несколько вариантов концепций), а еще лемматизировать или хотя бы использовать стемминг.

Когда будете сдавать это задание, пожалуйста, пришлите и файлы с текстами. И имейте в виду, если тексты будут вставлены прямо в код и слишком короткие, я задачу засчитаю только вполовину.

Напоминаю, как открываются файлы:

```
with open('путь к файлу - пишите прямые слеши', 'r', encoding='utf-8') as f:
    text = f.read() # все содержимое вашего файла считается в одну длинную строку
```

Настоятельно советую оформить код хотя бы в функции.

In [ ]:
!python -m spacy download ru_core_news_sm

In [ ]:
import spacy
nlp = spacy.load("ru_core_news_sm")

In [ ]:
from collections import Counter

def get_common_lemmas(txt_path, count_lemmas=30):
  with open(txt_path, 'r', encoding='utf-8') as f:
    text = f.read()
  doc = nlp(text)
  filtered_tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
  return Counter(filtered_tokens).most_common(count_lemmas)

def define_topic(path, themes_dict):
  explore_set = set(i[0] for i in get_common_lemmas(path))
  text_themes = dict()
  for key in themes_dict.keys():
    text_themes[key] = len(set(i[0] for i in themes_dict[key]) & explore_set)
  return text_themes, max(text_themes.items(), key = lambda x: x[1])

In [ ]:
themes_data = {'Шведская литература': '/content/Selma_Lagerlof.txt',
               'Химия': '/content/Liquid_crystals.txt',
               'Кавказ': '/content/South_Ossetia.txt',
               'История Руси': '/content/Batyi.txt',
               'Сказки': '/content/Fairy_tales.txt'
               }

In [ ]:
from pprint import pprint

themes_dict = dict()
for theme, path in themes_data.items():
  themes_dict[theme] = get_common_lemmas(path)
pprint(themes_dict)

{'История Руси': [('\xa0', 217),
                  ('\n', 24),
                  ('батый', 11),
                  ('князь', 9),
                  ('год', 8),
                  ('хан', 8),
                  ('город', 8),
                  ('юрий', 7),
                  ('русь', 6),
                  ('монгол', 6),
                  ('сын', 5),
                  ('кампания', 5),
                  ('часть', 5),
                  ('разорить', 5),
                  ('ордынец', 4),
                  ('удар', 4),
                  ('принять', 4),
                  ('осень', 4),
                  ('княжество', 4),
                  ('взять', 4),
                  ('всеволодович', 4),
                  ('киев', 4),
                  ('битва', 3),
                  ('дружина', 3),
                  ('поход', 3),
                  ('первый', 3),
                  ('полный', 3),
                  ('путь', 3),
                  ('монгольский', 3),
                  ('решение', 3)],
 'Кавказ': [('\n

In [ ]:
r = define_topic('/content/August_Strindberg.txt',themes_dict)
pprint(r[0])
f'Тема текста: {r[1][0]}, совпадений по частотным словам - {r[1][1]}'

{'История Руси': 1,
 'Кавказ': 6,
 'Сказки': 3,
 'Химия': 2,
 'Шведская литература': 7}


'Тема текста: Шведская литература, совпадений по частотным словам - 7'

#### Задача 2 (10 баллов).

Некоторые предлоги в русском языке могут управлять разными падежами (например, "я еду в Лондон" vs "я живу в Лондоне"). Давайте проанализируем эти предлоги и их падежи. Необходимо:

- составить список таких предлогов (РГ-80 вам в помощь)
- взять достаточно большой текст (можно большое художественное произведение)
- сделать морфоразбор этого текста
- Посчитать, как часто и какие падежи встречаются у слова, идущего после предлога.

Примечания: во-первых, имейте в виду, что иногда после предлога могут идти самые неожиданные вещи: "я что, должен ехать на, черт побери, северный полюс?". Во-вторых, неплохо бы учитывать отсутствие пунктуации (конечно, в норме, как нам кажется, предлог обязательно требует зависимое, но! "да иди ты на!") Эти штуки можно отсеять, если просто учитывать только заранее определенные падежи, а не считать все, какие встретились (так и None можно огрести).

Если будете использовать RNNMorph, возможно, понадобится регулярное выражение и немного терпения.

In [ ]:
import spacy
nlp = spacy.load("ru_core_news_sm")

In [24]:
from spacy.matcher import Matcher
from collections import defaultdict
import re

In [25]:
with open('/content/texts/Kuprin_Olesya.txt', 'r', encoding='utf-8') as f:
    text_kuprin = f.read()
doc_kuprin = nlp(text_kuprin)

matcher_1 = Matcher(nlp.vocab)
pattern_1 = [                               # шаблон только для простых предлогов
    {'POS': {'NOT_IN': ['ADP']}},           # исключение случая, когда перед предлогом еще токен с предлогом
    {'POS': 'ADP'},                         # {'POS': 'ADP', 'OP': '{3}'} - можно было бы использовать для парных предлогов-сращений 'из-за', 'из-под' и др.
    {'MORPH': {'REGEX': r'(?<=Case=)\w+'}}  # поиск последующих токенов с падежом (возможно упускаются случаи, описанные в условии)
]

matcher_1.add('pattern_acc', patterns=[pattern_1])
results_1 = matcher_1(doc_kuprin)

чтобы не проходиться циклом по доку использую [Matcher](https://spacy.io/api/matcher), итого дольше всего отрабатывает сам nlp(текста)

In [17]:
adp_dict = defaultdict(lambda: defaultdict(int))
for match_id, start, end in results_1:      # формирование словаря вида dict[ключ][внутренний ключ]
  adp_dict[str(doc_kuprin[end - 2]).lower()][re.search(r'(?<=Case=)\w+', str(doc_kuprin[end - 1].morph)).group(0)] += 1

sorted(adp_dict.items())

[('без', defaultdict(int, {'Gen': 13})),
 ('благодаря', defaultdict(int, {'Dat': 1})),
 ('в', defaultdict(int, {'Loc': 179, 'Acc': 113, 'Dat': 1, 'Nom': 1})),
 ('вблизи', defaultdict(int, {'Gen': 1})),
 ('ввиду', defaultdict(int, {'Gen': 2})),
 ('вдоль', defaultdict(int, {'Gen': 4})),
 ('вместо', defaultdict(int, {'Gen': 2})),
 ('во', defaultdict(int, {'Loc': 10, 'Acc': 5})),
 ('возле', defaultdict(int, {'Gen': 2})),
 ('вокруг', defaultdict(int, {'Gen': 4})),
 ('впереди', defaultdict(int, {'Gen': 1})),
 ('вроде', defaultdict(int, {'Gen': 2})),
 ('для', defaultdict(int, {'Gen': 22})),
 ('до', defaultdict(int, {'Gen': 29, 'Dat': 1})),
 ('за', defaultdict(int, {'Acc': 37, 'Ins': 13, 'Nom': 1})),
 ('из', defaultdict(int, {'Gen': 47})),
 ('изо', defaultdict(int, {'Gen': 1})),
 ('к', defaultdict(int, {'Dat': 73})),
 ('ко', defaultdict(int, {'Dat': 16})),
 ('кроме', defaultdict(int, {'Gen': 2})),
 ('между', defaultdict(int, {'Ins': 10, 'Gen': 1})),
 ('мимо', defaultdict(int, {'Gen': 1})),
 ('

Попытка составить список всех предлогов, [используя РГ-80](https://rkiff.philol.msu.ru/wp-content/uploads/2020/05/%D0%A0%D1%83%D1%81%D1%81%D0%BA%D0%B0%D1%8F-%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B0.-%D0%A8%D0%B2%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0.-%D0%A2%D0%BE%D0%BC-1.pdf), раздел "Предлоги" (§ 1655–1660)

**Первообразные**

род.п.: без безо для до из изо кроме меж между от ото ради с со у из-за из-под

дат.п: к ко по

вин.п.: в во за на о об обо по про с со через черезо

тв.п.: за меж между над надо перед передо пред предо с со по-за по-над

пред.п.: в во на о об обо по

**Простые наречные**

близ вблизи вглубь вдоль взамен вместо вне внутри внутрь возле вокруг вопреки впереди вроде вслед  касательно мимо наверху навстречу накануне наперекор  напротив около округ относительно поверх подле подобно позади помимо поперек после посреди посередине прежде против сбоку сверх свыше сзади сквозь согласно сообразно соответственно соразмерно среди

Большинство простых наречных предлогов сочетается с род. п., лишь несколько (вопреки, вслед, навстречу, наперекор, подобно, согласно,
 сообразно, соответственно и соразмерно) — с дат. п. и один (сквозь) — с вин. п.

**Составные наречные**

вблизи от, вдалеке от, вдали от, вместе с, вплоть до, впредь до, вровень с, вслед за, наравне с, наряду с, невдалеке от, независимо от, применительно к, рядом с, следом за, совместно с, согласно с, сообразно с, соответственно с, соразмерно с, сравнительно с

In [26]:
prep_txt = """без безо для до из изо кроме меж между от ото ради с со у из-за из-под к ко по в во за на о об обо по про с со через черезо за меж между над надо перед передо пред предо с со по-за по-над в во на о об обо по
близ вблизи вглубь вдоль взамен вместо вне внутри внутрь возле вокруг вопреки впереди вроде вслед касательно мимо наверху навстречу накануне наперекор напротив около округ относительно поверх подле подобно позади помимо поперек после посреди посередине прежде против сбоку сверх свыше сзади сквозь согласно сообразно соответственно соразмерно среди"""

sorted(set(prep_txt.split())) # часть из общего числа, но не исп. в коде, скорее для справки

['без',
 'безо',
 'близ',
 'в',
 'вблизи',
 'вглубь',
 'вдоль',
 'взамен',
 'вместо',
 'вне',
 'внутри',
 'внутрь',
 'во',
 'возле',
 'вокруг',
 'вопреки',
 'впереди',
 'вроде',
 'вслед',
 'для',
 'до',
 'за',
 'из',
 'из-за',
 'из-под',
 'изо',
 'к',
 'касательно',
 'ко',
 'кроме',
 'меж',
 'между',
 'мимо',
 'на',
 'наверху',
 'навстречу',
 'над',
 'надо',
 'накануне',
 'наперекор',
 'напротив',
 'о',
 'об',
 'обо',
 'около',
 'округ',
 'от',
 'относительно',
 'ото',
 'перед',
 'передо',
 'по',
 'по-за',
 'по-над',
 'поверх',
 'подле',
 'подобно',
 'позади',
 'помимо',
 'поперек',
 'посередине',
 'после',
 'посреди',
 'пред',
 'предо',
 'прежде',
 'про',
 'против',
 'ради',
 'с',
 'сбоку',
 'сверх',
 'свыше',
 'сзади',
 'сквозь',
 'со',
 'согласно',
 'сообразно',
 'соответственно',
 'соразмерно',
 'среди',
 'у',
 'через',
 'черезо']

In [23]:
import pandas as pd

df = pd.DataFrame.from_dict(adp_dict, orient='index')
df.fillna(0)

,Loc,Acc,Dat,Nom,Ins,Gen,Par
в,179.0,113.0,1.0,1.0,0.0,0.0,0.0
на,84.0,139.0,0.0,0.0,0.0,0.0,0.0
во,10.0,5.0,0.0,0.0,0.0,0.0,0.0
об,21.0,2.0,0.0,0.0,0.0,0.0,0.0
о,29.0,2.0,0.0,0.0,0.0,0.0,0.0
при,10.0,1.0,0.0,0.0,0.0,0.0,0.0
обо,6.0,0.0,0.0,0.0,0.0,0.0,0.0
с,0.0,5.0,2.0,0.0,183.0,24.0,2.0
за,0.0,37.0,0.0,1.0,13.0,0.0,0.0
под,0.0,6.0,0.0,0.0,16.0,1.0,0.0


[UD Case](https://universaldependencies.org/ru/feat/Case.html)

#### Задача 3 (5 баллов).

Представим, что у вас есть файл с разборами conllu (можете взять любой, например, [тут](https://github.com/dialogue-evaluation/GramEval2020)). Нужно просмотреть все примеры предложений с тегом dislocated и тегом discourse: напишите скрипт, который будет читать файл, находить все такие предложения и печатать: 1) сам текст предложения 2) слово, имеющее искомый тег. Если тег не был найден в файле, нужно об этом сообщить. Постарайтесь оформить вывод таким образом, чтобы это было удобно читать.

In [ ]:
!pip install conllu

In [128]:
from conllu import parse_incr

In [126]:
tags = ['dislocated', 'discourse']
results = []

data_file = open('/content/texts/social-dev.conllu', 'r', encoding='utf-8')
for sentence in parse_incr(data_file):
  sentence_text = ' '.join([token['form'] for token in sentence])
  for token in sentence:
    if token['deprel'] in tags:
      results.append({
          'Тег': token['deprel'],
          'Слово': token['form'],
          'Предложение': sentence_text
          })

if results:
    df = pd.DataFrame(results)
else:
    print('Не найдено предложений с указанными тегами.')

In [127]:
df

,Тег,Слово,Предложение
0,discourse,то,"Короче : столько - то "" я как Я "" , остальное ..."
1,discourse,Ой-ой-ой,"- Ой-ой-ой , а сам - то с мамой спишь каждый д..."
2,discourse,то,"- Ой-ой-ой , а сам - то с мамой спишь каждый д..."
3,discourse,Ну,"- Ну ты подожди ещё , у неё и дети будут ."
4,discourse,ну,И они на ковре вертолете сваливают сначала на ...


In [124]:
if results:
  pd.DataFrame(results)
else:
  print('Не найдено предложений с указанными тегами.')

#### Задача 4 (5 баллов).

Возьмите любой достаточно длинный (лучше новостной) текст. Любым известным инструментом извлеките именованные сущности из этого текста и выведите их списком по категориям (т.е. персоны вместе, локации вместе, организации вместе).

In [ ]:
!pip install natasha

In [130]:
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsNERTagger,
    Doc
)

In [139]:
with open('/content/texts/Sulak_news.txt', 'r', encoding='utf-8') as file:
    txt_news = file.read()

doc = Doc(txt_news)

In [141]:
segmenter = Segmenter()
emb = NewsEmbedding()

doc.segment(segmenter)
ner_tagger = NewsNERTagger(emb)
doc.tag_ner(ner_tagger)
doc.ner.print()

Бруно Сулак в конце 1970-х — начале 1980-х годов стал известен на всю 
PER────────                                                           
Францию. Несмотря на то что он ограбил большое количество 
LOC────                                                   
супермаркетов и ювелирных магазинов, ему удалось заслужить симпатии 
публики, прессы и даже некоторых полицейских. Он не применял насилие и
 забирал деньги только больших компаний. Не раз ему удавалось сбегать 
из-под ареста. Рассказываем историю французского Робин Гуда в 
                                                 PER───────   
еженедельной тру-крайм рубрике «Холода». 
В ночь с 17 на 18 марта 1985 года дверь в камеру известного грабителя 
Бруно Сулака в тюрьме Флери-Мерожи недалеко от Парижа открылась в 
PER─────────          LOC─────────             LOC───             
неположенное время. 
Появившийся в проеме молодой сотрудник тюремной администрации по имени
 Марк Метж вывел Сулака в коридор, где заключенного ждал замести

#### Задача на бонусные 5 баллов:

Сравните качество несколькиз разных морфопарсеров для любого языка, где их больше одного. Разберите этими морфопарсерами один и тот же текст, если они все разбирают в UD, можете вывести автоматически расхождения, в противном случае просмотрите глазами на наличие ошибок (текст, конечно, слишком большой лучше не брать).

Без письменных выводов-комментариев не засчитывается.

In [ ]:
# your code here